<a href="https://colab.research.google.com/github/AisyaChalvinaIzumi/AisyaChalvinaIzumi/blob/main/UAS_Big_Data_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [4]:
import os
os.environ["JAVA HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [5]:
!pip install -q findspark

In [94]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [85]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
sns.set(style="white")


import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [37]:
df = spark.read.csv("/imdb_top_1000.csv")

In [38]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)



In [45]:
df.show(15,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------+-------------+-----------+-------+-------------------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------+-----------------+--------------------+-----------------+----------------+-----------+-----------+
|_c0                                                                                                                                              |_c1                                              |_c2          |_c3        |_c4    |_c5                      |_c6        |_c7                                                                                            

In [40]:
df.count()

1001

In [49]:
df.select("_c1", "_c2", "_c5", "_c6", "_c15").show(15)

+--------------------+-------------+--------------------+-----------+-----------+
|                 _c1|          _c2|                 _c5|        _c6|       _c15|
+--------------------+-------------+--------------------+-----------+-----------+
|        Series_Title|Released_Year|               Genre|IMDB_Rating|      Gross|
|The Shawshank Red...|         1994|               Drama|        9.3| 28,341,469|
|       The Godfather|         1972|        Crime, Drama|        9.2|134,966,411|
|     The Dark Knight|         2008|Action, Crime, Drama|          9|534,858,444|
|The Godfather: Pa...|         1974|        Crime, Drama|          9| 57,300,000|
|        12 Angry Men|         1957|        Crime, Drama|          9|  4,360,000|
|The Lord of the R...|         2003|Action, Adventure...|        8.9|377,845,905|
|        Pulp Fiction|         1994|        Crime, Drama|        8.9|107,928,762|
|    Schindler's List|         1993|Biography, Drama,...|        8.9| 96,898,818|
|           Ince

In [78]:
df_filtered = df.filter("_c2 > 1980 AND _c6 > 7 AND _c1 IS NOT NULL")
df_filtered.select("_c1", "_c5", "_c6", "_c2").show(100, False)

+-------------------------------------------------+----------------------------+---+----+
|_c1                                              |_c5                         |_c6|_c2 |
+-------------------------------------------------+----------------------------+---+----+
|The Shawshank Redemption                         |Drama                       |9.3|1994|
|The Dark Knight                                  |Action, Crime, Drama        |9  |2008|
|The Lord of the Rings: The Return of the King    |Action, Adventure, Drama    |8.9|2003|
|Pulp Fiction                                     |Crime, Drama                |8.9|1994|
|Schindler's List                                 |Biography, Drama, History   |8.9|1993|
|Inception                                        |Action, Adventure, Sci-Fi   |8.8|2010|
|Fight Club                                       |Drama                       |8.8|1999|
|The Lord of the Rings: The Fellowship of the Ring|Action, Adventure, Drama    |8.8|2001|
|Forrest G

In [79]:
from pyspark.sql import functions as f
maxValue = df_filtered.agg(f.max("_c6")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("_c1","_c5", "_c6", "_c2").filter(df._c6 == maxValue).show(100, False)

maxValue:  9.3
+------------------------+-----+---+----+
|_c1                     |_c5  |_c6|_c2 |
+------------------------+-----+---+----+
|The Shawshank Redemption|Drama|9.3|1994|
+------------------------+-----+---+----+



In [65]:
df.filter("_c6="+str(df.agg(f.min("_c6")).collect()[0][0])).show()

+--------------------+--------------------+----+-----+-------+--------------------+---+--------------------+---+--------------------+-------------------+-------------------+----------------+--------------------+------+-----------+
|                 _c0|                 _c1| _c2|  _c3|    _c4|                 _c5|_c6|                 _c7|_c8|                 _c9|               _c10|               _c11|            _c12|                _c13|  _c14|       _c15|
+--------------------+--------------------+----+-----+-------+--------------------+---+--------------------+---+--------------------+-------------------+-------------------+----------------+--------------------+------+-----------+
|https://m.media-a...|         Dark Waters|2019|PG-13|126 min|Biography, Drama,...|7.6|A corporate defen...| 73|         Todd Haynes|       Mark Ruffalo|      Anne Hathaway|     Tim Robbins|        Bill Pullman| 60408|       null|
|https://m.media-a...|           Searching|2018|  U/A|102 min|Drama, Mystery

In [66]:
df.groupBy('_c2').count().sort(df._c2).show()

+----+-----+
| _c2|count|
+----+-----+
|1920|    1|
|1921|    1|
|1922|    1|
|1924|    1|
|1925|    2|
|1926|    1|
|1927|    2|
|1928|    2|
|1930|    1|
|1931|    3|
|1932|    2|
|1933|    3|
|1934|    2|
|1935|    3|
|1936|    1|
|1937|    1|
|1938|    3|
|1939|    5|
|1940|    7|
|1941|    2|
+----+-----+
only showing top 20 rows

